In [ ]:
# Install required libraries
!pip install requests pandas

import requests
import pandas as pd
import time

# === Step 1: Define activist keywords ===
activist_keywords = [
    "centralize", "centralization", "restructure", "terminate", "change", "adjust",
    "re-assign", "reassign", "revoke", "unvested", "vesting", "vest immediately",
    "grant misuse", "misuse", "spot and report", "accountability", "hold accountable",
    "alter governance", "pivot", "delegate removal", "adjust voting power", "modify voting",
    "treasury", "funding", "allocate", "redistribute", "reclaim", "withdraw", "return funds",
    "parameter change", "steward", "control", "power", "revise", "defend", "legal threat",
    "policy operation", "DAO control", "redeploy", "ratify", "redeem", "proposal to change",
    "governance shift", "fund delegation", "facilitator", "watchdog", "proposal to modify"
]

# === Step 2: Get all Snapshot spaces ===
def fetch_spaces():
    url = "https://hub.snapshot.org/graphql"
    query = """
    {
      spaces(first: 1000) {
        id
      }
    }
    """
    response = requests.post(url, json={"query": query})
    data = response.json()
    return [space["id"] for space in data["data"]["spaces"]]

# === Step 3: Fetch proposals for each space ===
def fetch_proposals(space, first=100):
    url = "https://hub.snapshot.org/graphql"
    query = """
    query Proposals($space: String!, $first: Int!) {
      proposals(first: $first, where: { space: $space }) {
        id
        title
        body
        start
        end
        state
        choices
        scores
      }
    }
    """
    variables = {"space": space, "first": first}
    response = requests.post(url, json={"query": query, "variables": variables})
    if response.status_code != 200:
        return []
    try:
        return response.json()["data"]["proposals"]
    except:
        return []

# === Step 4: Fetch voters and their voting power ===
def fetch_votes(proposal_id):
    url = "https://hub.snapshot.org/graphql"
    query = """
    query Votes($proposal_id: String!) {
      votes(first: 1000, where: {proposal: $proposal_id}) {
        voter
        vp
        vp_by_strategy
        created
      }
    }
    """
    response = requests.post(url, json={
        "query": query,
        "variables": {"proposal_id": proposal_id}
    })
    if response.status_code != 200:
        return []
    try:
        return response.json()["data"]["votes"]
    except:
        return []

# === Step 5: Main loop ===
all_results = []
dao_spaces = fetch_spaces()
print(f"Found {len(dao_spaces)} Snapshot spaces. Fetching proposals...")

for i, dao in enumerate(dao_spaces):
    print(f"[{i+1}/{len(dao_spaces)}] Fetching: {dao}")
    proposals = fetch_proposals(dao)

    for p in proposals:
        title = p["title"].lower()
        body = p["body"].lower() if p["body"] else ""
        if any(keyword in title or keyword in body for keyword in activist_keywords):
            total_votes = sum(p["scores"]) if p["scores"] else 0

            # Fetch votes
            top_voter = None
            top_voter_vp = 0
            votes = fetch_votes(p["id"])
            if votes:
                top_vote = max(votes, key=lambda x: x["vp"])
                top_voter = top_vote["voter"]
                top_voter_vp = top_vote["vp"]

            all_results.append({
                "DAO": dao,
                "Proposal ID": p["id"],
                "Title": p["title"],
                "Body": p["body"][:200],
                "Start (Unix)": p["start"],
                "End (Unix)": p["end"],
                "State": p["state"],
                "Choices": p["choices"],
                "Total Votes (score sum)": total_votes,
                "Top Voter": top_voter,
                "Top Voter VP": top_voter_vp
            })

    time.sleep(0.3)  # Avoid rate-limiting

# === Step 6: Export to CSV ===
df = pd.DataFrame(all_results)
df.to_csv("activist_snapshot_proposals_with_voters.csv", index=False)
print(f"\n✅ Done! {len(df)} activist proposals saved to 'activist_snapshot_proposals_with_voters.csv'.")

# Show preview
df.head()


Found 1000 Snapshot spaces. Fetching proposals...
[1/1000] Fetching: yetavahum.eth
[2/1000] Fetching: shafiwong.eth
[3/1000] Fetching: nfd.eth
[4/1000] Fetching: basedbrew.eth
[5/1000] Fetching: meta-whale-ai.eth
[6/1000] Fetching: zamm.eth
[7/1000] Fetching: peter127.base.eth
[8/1000] Fetching: apollondao.eth
[9/1000] Fetching: kaistdao.eth
[10/1000] Fetching: daolab.eth
[11/1000] Fetching: gptdatadao.eth
[12/1000] Fetching: web3classdao.eth
[13/1000] Fetching: mega06.eth
[14/1000] Fetching: believee.eth
[15/1000] Fetching: blockchainmvn.eth
[16/1000] Fetching: claude13.eth
[17/1000] Fetching: atlantisonsonic.eth
[18/1000] Fetching: mountain-retreat-villa.eth
[19/1000] Fetching: zhenglong.eth
[20/1000] Fetching: nullz.base.eth
[21/1000] Fetching: kilt-protocol.eth
[22/1000] Fetching: keienwang.eth
[23/1000] Fetching: 7cedars.eth
[24/1000] Fetching: crowdraise.eth
[25/1000] Fetching: amircryb.eth
[26/1000] Fetching: csigmadao.eth
[27/1000] Fetching: leviathannews.eth
[28/1000] Fetching

,DAO,Proposal ID,Title,Body,Start (Unix),End (Unix),State,Choices,Total Votes (score sum),Top Voter,Top Voter VP
0,basedbrew.eth,0x54648f5dd67aeb130bb21dded1cfa9a0869d38ff1a5b...,Determination of NFT Staking strategy for Brew...,This proposal is to determine sentiment from B...,1752101729,1753224900,active,"[Option A, Option B, Option C]",4.920000e+01,0x4b34C7EedF0358c4ccda9d054a54fF465E3e575d,2.350000e+01
1,meta-whale-ai.eth,0xbfe9292417de5725a86608a5310c92acbd5651459b76...,Strategic vote on the CES unlock mode,We present five possible scenarios for unlocki...,1752577200,1752836400,pending,"[2% per month, 3% per month, 5% per month, 7% ...",0.000000e+00,None,0.000000e+00
2,zamm.eth,0xbaa757c6d1582374ad60e6b72984903e56d3a1f3f072...,Turn on ZAMM Protocol Fee Switch,This proposal seeks consensus from [ZAMM](http...,1752060050,1752319250,closed,"[For, Against, Abstain]",5.277858e+05,0x21caDed0E50b60646af532E37D3124cF2669752B,4.129543e+05
3,gptdatadao.eth,0x1db84cfbf7f689aa44adc82cf016c2fda5dc45858939...,Establishing Snapshot as the Official Governan...,# Governance Proposal: Snapshot Platform for G...,1751932907,1752192107,closed,"[For, Against, Abstain]",2.600000e+01,0x1A5e52d80B7d3F127cedeBeaE70D4A4056BC2626,2.400000e+01
4,csigmadao.eth,0xa02f697af90baa3d4949c4271079475c5bd6aee9a927...,CSP-001: SIGMA Token Staking APR Selection,**Synopsis** \nThe cSigma community will deter...,1751481000,1752172140,closed,"[3% APR, 3.5% APR, 4% APR, 4.5% APR]",8.268062e+06,0xd0e475d1DA95D9C7fAF90c3d91B8F23CE0e99aDf,8.266744e+06


In [ ]:
!pip install pycoingecko pandas openpyxl

from pycoingecko import CoinGeckoAPI
import pandas as pd
from datetime import datetime, timedelta
import time

cg = CoinGeckoAPI()

# === Function to get daily price data for a token around a proposal date ===
def get_price_data(token_id, start_date, end_date):
    from_timestamp = int(start_date.timestamp())
    to_timestamp = int(end_date.timestamp())

    data = cg.get_coin_market_chart_range_by_id(
        id=token_id,
        vs_currency='usd',
        from_timestamp=from_timestamp,
        to_timestamp=to_timestamp
    )

    prices = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    prices['date'] = pd.to_datetime(prices['timestamp'], unit='ms').dt.date
    prices = prices[['date', 'price']]
    return prices

# === Main Function to Generate Excel with Price Data ===
def generate_price_spreadsheet(file_name):
    df = pd.read_csv(file_name)

    # Column 0 = DAO name (CoinGecko ID), Column 5 = End date
    token_ids = df.iloc[:, 0]
    proposal_dates = df.iloc[:, 5]

    writer = pd.ExcelWriter("price_data_output.xlsx", engine='openpyxl')

    for i in range(len(df)):
        token_id = token_ids[i]
        proposal_date = proposal_dates[i]

        if pd.isna(token_id) or pd.isna(proposal_date):
            continue

        try:
            proposal_datetime = pd.to_datetime(proposal_date) # Convert to datetime
            start_date = proposal_datetime - timedelta(days=60)
            end_date = proposal_datetime + timedelta(days=60)

            price_df = get_price_data(token_id, start_date, end_date)

            if not price_df.empty: # Only write if there is data
                sheet_name = f"{token_id[:25]}_{i}"  # Excel max sheet name = 31 chars
                price_df.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"✅ Fetched: {token_id} | Proposal Date: {proposal_date}")
                time.sleep(1.2)  # Respect CoinGecko API rate limit
            else:
                print(f"⚠️ No price data found for {token_id} around {proposal_date}")


        except Exception as e:
            print(f"❌ Error for {token_id} at row {i}: {e}")

    writer.close()
    print("✅ All done. Saved to price_data_output.xlsx")


generate_price_spreadsheet("activist_snapshot_proposals_with_voters.csv")

❌ Error for basedbrew.eth at row 0: {'error': 'Incorrect path. Please check https://docs.coingecko.com/'}
❌ Error for meta-whale-ai.eth at row 1: {'error': 'Incorrect path. Please check https://docs.coingecko.com/'}
❌ Error for zamm.eth at row 2: {'error': 'Incorrect path. Please check https://docs.coingecko.com/'}
❌ Error for gptdatadao.eth at row 3: {'error': 'Incorrect path. Please check https://docs.coingecko.com/'}
❌ Error for csigmadao.eth at row 4: {'error': 'Incorrect path. Please check https://docs.coingecko.com/'}
❌ Error for leviathannews.eth at row 5: {'error': 'Incorrect path. Please check https://docs.coingecko.com/'}
❌ Error for leviathannews.eth at row 6: {'error': 'Incorrect path. Please check https://docs.coingecko.com/'}
❌ Error for leviathannews.eth at row 7: {'error': 'Incorrect path. Please check https://docs.coingecko.com/'}
❌ Error for lumiterra.eth at row 8: {'error': 'Incorrect path. Please check https://docs.coingecko.com/'}
❌ Error for lumiterra.eth at row 9

KeyboardInterrupt: 